In [ ]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)

In [ ]:
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json


#KAFKA variables, Move to the OS variables or configuration
# This will work in local Jupiter Notebook, but in a databrick, hiding config.py is tougher. 
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
confluentTopicName = "Group3Data"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "YHMHG7E54LJA55XZ"
confluentSecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistryApiKey = "YHMHG7E54LJA55XZ"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"


#Kakfa Class Setup.
c = Consumer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,# this will create a new consumer group on each invocation.
    'group.id': str(1),
    'auto.offset.reset': 'earliest',
    'enable.auto.commit': True,
    'error_cb': error_cb,
})

c.subscribe(['Group3Data'])

In [ ]:
aString = {}

kafkaList = []

for i in range(1000):
    try:
        msg = c.poll(timeout=15)
        print(msg)
        if msg is None:
            break
        elif msg.error():
            print("Consumer error: {}".format(msg.error()))
            break
        else:
            aString=json.loads('{}'.format(msg.value().decode('utf-8')))
            print('#' + str(i) + ' ' + str(aString))
            kafkaList.append(aString)
            c.commit(asynchronous=False)
    except Exception as e:
        print(e)

In [ ]:
print(kafkaList)


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(kafkaList)

df.info()

In [ ]:
finaldf = spark.createDataFrame(df)
finaldf.dtypes

In [ ]:
from pyspark.sql.types import DecimalType, StringType, FloatType

output_df = finaldf \
  .withColumn("DistrictName", finaldf["DistrictName"].cast(StringType())) \
  .withColumn("GraduationPercentageRate", finaldf["GraduationPercentageRate"].cast(FloatType())) \
  .withColumn("DropoutPercentageRate", finaldf["DropoutPercentageRate"].cast(FloatType())) \
  .withColumn("FundingPerStudent", finaldf["FundingPerStudent"].cast(FloatType())) \
  .withColumn("TotalEnrolled", finaldf["TotalEnrolled"].cast(FloatType())) \
  .withColumn("MalesEnrolled", finaldf["MalesEnrolled"].cast(FloatType())) \
  .withColumn("FemalesEnrolled", finaldf["FemalesEnrolled"].cast(FloatType())) \
  .withColumn("Percentage of Minority Students", finaldf["Percentage of Minority Students"].cast(FloatType())) \
  .withColumn("HouseholdIncome", finaldf["HouseholdIncome"].cast(FloatType())) \
  .withColumn("FederallyFunded", finaldf["FederallyFunded"].cast(StringType())) 

In [ ]:
output_df = output_df.withColumnRenamed("Percentage of Minority Students", "PercentageofMinorityStudents")
output_df.dtypes

In [ ]:
database = "Capstone-Group3"
table = "dbo.nystatedistricts"
user = "stanleyperez"
password  = ""
server = "gen10-data-fundamentals-22-05-sql-server.database.windows.net"

output_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("overwrite") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()